# **Cracking the Code: Unlocking Success in Trading with Reinforcement Learning**


This project focuses on applying reinforcement learning techniques to develop an automated stock trading system. By leveraging technical indicators and a defined reward function, the project aims to train an intelligent agent capable of making informed decisions on buying, selling, or holding stocks. The project involves preprocessing stock data, defining the trading environment, training the reinforcement learning agent using algorithms like Deep Q-Network, and evaluating its performance on separate datasets. Through this project, the goal is to create a robust model that can adapt to changing market conditions and optimize trading strategies for potential profit.

| <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMSkillsNetwork-GPXX0SQEN/images/wance-paleri-HPM0CDfNtx0-unsplash.jpg" width="400" alt="trading; image credit: https://unsplash.com/"> |
|:--:| 


## __Table of Contents__

<ol>
    <li><a href="#Objectives">Objectives</a></li>
    <li>
        <a href="#Setup">Setup</a>
        <ol>
            <li><a href="#Installing-Required-Libraries">Installing Required Libraries</a></li>
            <li><a href="#Importing-Required-Libraries">Importing Required Libraries</a></li>
        </ol>
    </li>
    <li>
        <a href="#Indicators">Types of Indicators</a>
        <ol>
            <li><a href="#">Trend Indicators</a></li>
            <li><a href="#">Momentum Indicators</a></li>
            <li><a href="#">Volume Indicators</a></li>
            <li><a href="#">Volatility Indicators</a></li>
        </ol>
    </li>
    <li><a href="#">Reinforcement Learning</a></li>
    <li><a href="#">Defining Environment</a></li>
    <li><a href="#">Defining Agent</a></li>
    <li><a href="#">Training Agent</a></li>
    <li><a href="#">Testing Agent</a></li>
    <li><a href="#">Tips on Tuning Agent</a></li>
</ol>

<a href="#Practice on more data">Practice on More Data</a>


----


## Setup


### Installing Required Libraries


The following required libraries are __not__ pre-installed in the Skills Network Labs environment. __You will need to run the following cell__ to install them:


In [1]:
!pip install gym yfinance ta

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 721.7/721.7 kB 45.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.3/65.3 kB 9.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.6/62.6 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 16.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.2/112.2 kB 19.7 MB/s eta 0:00:00
  Created wheel for gym: filename=gym-0.26.2-py3-none-any.whl size=827620 sha256=1dcc541a6ed8980f7cfb6fd54f6143901cee0319b8274ea2f9e9b7e552e202e8
  Stored in directory: /home/jupyterlab/.cache/pip/wheels/4d/c8/dc/d08577bffa680f083f04448e81e4f176d0e45a13eb16f93c0c
  Created wheel for ta: filename=ta-0.10.2-py3-none-any.whl size=29088 sha256=b661dbf5d31278daf4605eca909f5c8647440749d94ee5a83a091ec5a00bd6a9
  Stored in di

### Importing Required Libraries

_We recommend you import all required libraries in one place (here):_


In [ ]:
# You can also use this section to suppress warnings generated by your code:
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn
warnings.filterwarnings('ignore')

import gym
import yfinance as yf
import ta   # ta documentation: https://technical-analysis-library-in-python.readthedocs.io/en/latest/
from tqdm import tqdm
import numpy as np
import pandas as pd
from itertools import accumulate
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline


sns.set_style("darkgrid")

--------


## Look a head:
We are aiming to implement reinforcement learning system for stock trading using indicators:

Step 1: Data preparation
- Preprocess the stock dataset by cleaning and add indicators.

Step 2: Define the environment 
- Define the action space, which represents the actions the agent can take. In this case, the agent can either buy or sell the stock.
- Define the state space, which represents the current state of the stock. In this case, the state space can include the current price, the indicators.
- Define the reward function, which will be used to evaluate the agent's performance. In this case, the reward function can be based on the profit or loss made by the agent.

Step 3: Define the agent
- Define the agent, which will take actions based on the current state of the stock and the reward function.
- Choose a reinforcement learning algorithm such as Q-Learning or Deep Q-Network (DQN) to train the agent.

Step 4: Train the agent
- Train the agent using the stock data and the defined reward function.
- The agent should learn the optimal policy for buying and selling the stock based on the current state of the stock and the reward function.

Step 5: Test the agent
- Test the agent on a separate dataset to evaluate its performance.

Step 6: Refine the model
- Refine the model by adjusting the parameters of the reinforcement learning algorithm, the reward function, or the state space.

------


## Loading the Data



Using Yahoo Finance python library (`yfinance`), we are able to retrieve live data in each call.

In the example below, we obtain hourly data of S&P500 (the ticker is `^GSPC`) in the period of 2 years. You can change the ticker to `AAPL` to obtain Apple stock prices.


In [ ]:
tickers = "^GSPC"  #"^GSPC" S&P500
df = yf.download(tickers = tickers ,       # list of tickers
                  period = "2y",         # time period
                  interval = "1h",       # trading interval
                  ignore_tz = True,      # ignore timezone when aligning data from different exchanges?
                  prepost = False)       # download pre/post market hours data?  
print("shape of dataset: ",df.shape)
df.head()

You can see that the `yfinance` library returns a dataframe with 6 columns. To plot the price, we have selected the `Adj Close` column, which represents the adjusted price taking into account any stock splits that may have occurred prior to the specified date. We specify the `split_line` which is the point of separating data into Training and Testing.


In [ ]:
split_line = int(np.round(len(df)* 0.1)) # use 10% of data for testing

In [ ]:
# Get min and max of 'Adj Close'
adj_close_min = df['Adj Close'].min()
adj_close_max = df['Adj Close'].max()

# Set the background color
sns.set_style(rc = {'axes.facecolor': 'lightsteelblue'})
plt.figure(figsize=(12,4))

# Create line plot
sns.lineplot(data=df["Adj Close"],label = "Adjusted Close Price")
plt.xticks(rotation=45)
plt.title(tickers)

# single vline with full ymin and ymax
plt.vlines(x=df.index[-split_line], ymin=adj_close_min, ymax=adj_close_max, colors='green', ls=':', lw=2, label='Train-Test Split Point')

plt.legend(bbox_to_anchor=(1.0, 1), loc='upper left')
plt.show()


## Step 1: Data Preparation (Adding Trading Indicators)


Trading indicators are tools used by traders to analyze market data and make informed trading decisions. There are several types of trading indicators, each with unique characteristics and applications. Here are five types of trading indicators with an example for each:
  

1. Momentum Indicators
2. Volume Indicators
3. Volatility Indicators
4. Trend Indicators
5. Support and Resistance


Remember that no single indicator is perfect, and traders often use a combination of indicators to make more informed decisions. It's essential to understand the strengths and limitations of each indicator and apply them in the context of your trading strategy and risk management.

> ___The Good trading strategy is the one that employs all types of market behaviour (all types of indicator)___


### Trend Indicators 

These indicators help traders identify the direction and strength of a market trend. They typically measure the asset's price movement over time and help traders determine if a trend is bullish (upward) or bearish (downward).

Example: Moving Averages (MA)
Moving averages smooth out price data by calculating the average price of an asset over a specified time period. Popular types of moving averages include the Simple Moving Average (SMA) and the Exponential Moving Average (EMA). Traders often use moving averages to identify trends and potential entry/exit points.


#### EMAs : Exponential moving averages


In [ ]:
df['EMA7'] = ta.trend.EMAIndicator(close= df['Adj Close'], window= 7, fillna= False).ema_indicator()
df['EMA14'] = ta.trend.EMAIndicator(close= df['Adj Close'], window= 14, fillna= False).ema_indicator()
df['EMA50'] = ta.trend.EMAIndicator(close= df['Adj Close'], window= 50, fillna= False).ema_indicator()
df['EMA200'] = ta.trend.EMAIndicator(close= df['Adj Close'], window= 50, fillna= False).ema_indicator()

Plotting the EMAs.


In [ ]:
# Set the background color
sns.set_style(rc = {'axes.facecolor': 'lightsteelblue'})
plt.figure(figsize=(12,4))

# Create line plot
sns.lineplot(data=df["Adj Close"],label = "Adjusted Close Price")
sns.lineplot(data=df["EMA7"],label = "EMA7")
sns.lineplot(data=df["EMA14"],label = "EMA14")
sns.lineplot(data=df["EMA50"],label = "EMA50")
sns.lineplot(data=df["EMA200"],label = "EMA200")


plt.xticks(rotation=45)
plt.title(tickers)

### Momentum Indicators 

These indicators measure the strength and rate of change in price movements. They help traders identify potential turning points in the market and can indicate whether an asset is overbought or oversold.

Example: Relative Strength Index (RSI) and MACD
The RSI is a popular momentum oscillator that measures the speed and change of price movements on a scale of 0-100. An RSI reading below 30 typically indicates that an asset is oversold, while a reading above 70 suggests that it is overbought.

The Moving Average Convergence Divergence (MACD) is a momentum indicator. It helps traders identify trend direction, potential trend reversals, and the strength of price movements. The MACD is calculated by subtracting a longer-term Exponential Moving Average (EMA) from a shorter-term EMA, usually the 26-day and 12-day EMAs, respectively. A signal line, typically the 9-day EMA of the MACD, is also plotted on the same chart.



 
 | <img src="https://commodity.com/wp-content/uploads/technical-analysis/MACDbasicsQQQQ.gif" width="500" alt="MACD">|
|:--:| 
| *An example of MACD, image credit: commodity.com*  |


In [ ]:
df['MACD_line'] =ta.trend.MACD(close= df['Adj Close'], window_slow= 26, window_fast= 12, window_sign = 9, fillna = False).macd()
df['MACD_signal'] =ta.trend.MACD(close= df['Adj Close'], window_slow= 26, window_fast= 12, window_sign = 9, fillna = False).macd_signal()
df['MACD_diff'] =ta.trend.MACD(close= df['Adj Close'], window_slow= 26, window_fast= 12, window_sign = 9, fillna = False).macd_diff()

#### RSI: Relative Strength Index 




| <img src="https://www.investopedia.com/thmb/lXq2-HwfRcFpe_0NsEieuTXXaUw=/1500x0/filters:no_upscale():max_bytes(150000):strip_icc()/dotdash_final_Relative_Strength_Index_RSI_Jul_2020-01-98fcd0c032cb4f0da005c31c44440c90.jpg" width="500" alt="RSI, credit: www.investopedia.com">|
|:--:| 
| An example of RSI indicator *image credit: https://pixabay.com/* |


In [ ]:
df['RSI']=ta.momentum.RSIIndicator(close= df['Adj Close'], window= 14, fillna= False).rsi()

### Volume Indicators 

> These indicators analyze the trading volume to assess the strength and significance of price movements. High volume can confirm the validity of a trend or signal a reversal.

> On-Balance Volume (OBV) is a cumulative indicator that adds volume on up days and subtracts volume on down days. It helps traders identify buying or selling pressure in the market. A rising OBV indicates strong buying pressure, while a falling OBV signals strong selling pressure.
4. Volatility Indicators: These indicators measure the degree of price fluctuations and help traders gauge market sentiment and potential price breakouts.


In [ ]:
# adding Volumn indicator
df['OBV']= ta.volume.OnBalanceVolumeIndicator(close= df['Adj Close'], volume= df['Volume'], fillna = False).on_balance_volume()
df.head()

### Volatility Indicators
> These indicators measure the degree of price fluctuations and help traders gauge market sentiment and potential price breakouts.

> Bollinger Bands consist of a centerline (typically an SMA) and two outer bands, which are standard deviations above and below the centerline. The bands expand and contract based on market volatility. When the bands are narrow, volatility is low, and when they widen, volatility is high. Traders use Bollinger Bands to identify potential entry/exit points and overbought/oversold conditions.


| <img src="https://a.c-dn.net/b/3u3xku/trading-forex-with-bollinger-bands_body_Mainimage.png" width="50%" alt="BB image credit:https://www.dailyfx.com/">|
|:--:| 
| An example of MACD indicator *image credit: https://www.dailyfx.com/* |


In [ ]:
df['BBH']=ta.volatility.BollingerBands(df['Adj Close'], window = 20, window_dev = 2, fillna = False).bollinger_hband_indicator()
df['BBL']=ta.volatility.BollingerBands(df['Adj Close'], window = 20, window_dev = 2, fillna = False).bollinger_lband_indicator()
df.head()

In [ ]:
df.shape

In [ ]:
df.columns

In [ ]:
df=df.dropna()
df = df.drop(columns=['Open', 'High', 'Low', 'Close', 'Volume'],axis=1)
df.head()

In [ ]:
train_data = df.iloc[:-split_line,:]
test_data = df.iloc[-split_line:,:]
input_shape= train_data.shape[1]
print(input_shape)
print('Training data:',train_data.shape)
print('Testing data',test_data.shape)

## Step 2: Define the environment




|  |  |
|-----|---|
|    | **Environment**  |
| <img style="float: left;" src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMSkillsNetwork-GPXX0SQEN/images/_2800865b-9afe-411a-9d9c-80e322f466b3.jpeg" width="900" alt="environment"> | <span style="font-family: Verdana, sans-serif; font-size: 15px;"> The `StockTradingEnv` is a custom environment for stock trading, which extends the `gym.Env` class from OpenAI's Gym library. The environment is initialized with stock data containing daily stock prices and various technical indicators. The environment's purpose is to simulate a stock trading scenario where an agent can decide to buy or sell shares based on the provided market data.</span> <br>  <br> The main components of the environment are: <br> - Initialization <br> - Reset <br> - Step <br> - Get observation |





1. **Initialization** : The environment is initialized with stock data and defines the action and observation spaces. The action space is discrete, with two possible actions (0: Buy, 1: Sell). The observation space is continuous, consisting of the stock's closing price and the values of various technical indicators at the current time step. 
    - In the  code, `self.action_space = gym.spaces.MultiDiscrete([3, 10])` defines the action space for the StockTradingEnv environment. `gym.spaces.MultiDiscrete` is a class from the OpenAI Gym library that is used to define a multidimensional discrete action space. It accepts a list of integers, where each integer specifies the number of possible values for a particular dimension. In this case, [3, 10] is passed to the MultiDiscrete class, creating a 2D action space with the following properties: The first dimension has 3 possible values (0, 1, 2), which represent different action types: buy (0), sell (1), or hold (2). The second dimension has 10 possible values (0, 1, 2, ..., 9), representing the number of shares to trade during the buy or sell action.
    
2. **Reset** : The `reset` method resets the environment to its initial state, setting the current step to 0, resetting profits, shares held, and net worth, and returning the initial observation. 
3. **Step** : The `step` method takes in an action and updates the environment state accordingly. If the action is to buy, the environment calculates the number of shares that can be bought with the available cash, deducts the purchase price from the cash balance, and adds the purchased shares to the shares held variable. If the action is to sell, the environment sells the specified number of shares, adds the sale price to the cash balance, and calculates the profits made based on the difference between the purchase and sale prices. The method returns the updated observation, profits, done flag, and an empty dictionary.
4. **Get observation** : The `_get_obs` method returns the current observation, which is a numpy array containing the stock's closing price and the values of various technical indicators at the current time step.

The environment provides a framework for training reinforcement learning agents in a stock trading scenario by allowing them to take actions (buy or sell shares) based on the available market data and learn from the resulting profits.




In [ ]:
import gym
import numpy as np

class StockTradingEnv(gym.Env):
    def __init__(self, data, initial_cash=10000, commission=0.000):
        # Define the action space as a 2D vector representing the action type (buy or sell) and number of shares to trade
        self.action_space = gym.spaces.MultiDiscrete([3, 10])  # [action, number of shares]

        # Define the observation space as a X-D vector containing various indicators of the stock price
        self.observation_space = gym.spaces.Box(low=-np.inf, high=np.inf, shape=(12,), dtype=np.float32)

        # Store the historical stock data, initial cash balance, and commission rate as instance variables
        self.data = data
        self.initial_cash = initial_cash
        self.commission = commission
        self.end_step = len(self.data) - 1

        # Reset the environment to its initial state
        self.reset()

    def reset(self):
        # Set the current step to 0 and reset the state variables
        self.current_step = 0
        self.profits = 0
        self.shares_held = 0
        self.cash = self.initial_cash
        self.buy_price = 0
        self.sell_price = 0
        self.done = False

        # Return the initial observation
        return self._get_obs()

    def step(self, action):
        action_type, amount = action

        if action_type == 0:
            cost = self.data['Adj Close'][self.current_step] * amount
            if self.cash >= cost:
                self.cash -= cost
                self.shares_held += amount
                self.buy_price = self.data['Adj Close'][self.current_step]
        elif action_type == 1:
            shares_sold = min(self.shares_held, amount)
            revenue = self.data['Adj Close'][self.current_step] * shares_sold
            self.cash += revenue
            self.profits += (self.data['Adj Close'][self.current_step] - self.buy_price) * shares_sold
            self.shares_held -= shares_sold
            self.sell_price = self.data['Adj Close'][self.current_step]

        self.current_step += 1

        # Calculate the total portfolio value
        portfolio_value = self.cash + (self.shares_held * self.data['Adj Close'][self.current_step])
  
        # Calculate the reward based on the change in portfolio value at each stept
        reward = portfolio_value - (self.cash + (self.shares_held * self.data['Adj Close'][self.current_step - 1]))
        done = (self.current_step == self.end_step)
        return self._get_obs(), reward, done, {}


    def _get_obs(self):
    # Calculate the observation vector for the current step
        obs = np.array([
            self.data['Adj Close'][self.current_step],
            self.data['EMA7'][self.current_step],
            self.data['EMA14'][self.current_step],
            self.data['EMA50'][self.current_step],
            self.data['EMA200'][self.current_step],
            self.data['MACD_line'][self.current_step],
            self.data['MACD_signal'][self.current_step],
            self.data['MACD_diff'][self.current_step],
            self.data['RSI'][self.current_step],
            self.data['OBV'][self.current_step],
            self.data['BBH'][self.current_step],
            self.data['BBL'][self.current_step],
       
            
            
            
            
            
        ])

        # Return the observation vector
        return obs


<details>
    <summary>Click here for Solution</summary>
    
```python
    def _get_obs(self):
    # Calculate the observation vector for the current step
        obs = np.array([
            self.data['Adj Close'][self.current_step],
            self.data['EMA7'][self.current_step],
            self.data['EMA14'][self.current_step],
            self.data['EMA50'][self.current_step],
            self.data['EMA200'][self.current_step],
            self.data['MACD_line'][self.current_step],
            self.data['MACD_signal'][self.current_step],
            self.data['MACD_diff'][self.current_step],
            self.data['RSI'][self.current_step],
            self.data['OBV'][self.current_step],
            self.data['BBH'][self.current_step],
            self.data['BBL'][self.current_step],
        ])
```
</details>



___We defines a custom OpenAI gym environment for simulating a simple stock trading scenario, and implements the necessary methods for the reinforcement learning agent to interact with the environment. The observation and action spaces, as well as the reward function, can be customized to suit different trading strategies and datasets.___


## Step 3: Define The Agent
| <img src="https://cdn.pixabay.com/photo/2019/01/22/10/58/pixel-cells-3947912_1280.png" width="200" alt="Cyber attack image"> |
|:--:| 
| *image credit: https://pixabay.com/* |



The `DQNAgent` class defines a deep Q-learning agent that uses a neural network to approximate the Q-function. This agent learns to make decisions in the stock trading environment based on the market data provided in the environment's observations.

The agent's main components are described as follows:


<div style="display: flex; align-items: center;">
  



1. **Initialization**: The agent is initialized with the state size, action space, and various learning parameters, such as gamma, epsilon, epsilon decay, and learning rate. The agent also initializes a neural network model using the `_build_model` method.
2. **Build Model**: The `_build_model` method defines a neural network architecture using TensorFlow's Keras API. The network consists of two hidden layers with 64 neurons each and ReLU activation functions. The output layer has a linear activation function, and the number of output neurons is equal to the size of the action space. The model is compiled using the mean squared error loss function and the Adam optimizer.
3. **Remember**: The `remember` method stores the agent's experiences (state, action, reward, next state, done) in a memory buffer, implemented as a deque with a maximum length of 2000. This buffer is used to sample experiences for training the agent's neural network.
4. **Act**: The `act` method selects an action based on the agent's current state. With probability epsilon, the agent selects a random action, and with probability 1 - epsilon, the agent selects the action with the highest Q-value predicted by the neural network. The agent's epsilon value decays over time to balance exploration and exploitation.
5. **Replay**: The `replay` method samples a batch of experiences from the memory buffer and uses them to train the agent's neural network. The agent computes target Q-values for the selected actions based on the rewards and discounted future Q-values of the next states. The agent updates the neural network weights by minimizing the mean squared error between the predicted Q-values and the target Q-values.


</div>

<br>

The `DQNAgent` class provides a framework for training a deep reinforcement learning agent to make buy and sell decisions in the stock trading environment using deep Q-learning. The agent learns to approximate the Q-function through a neural network and balances exploration and exploitation using an epsilon-greedy action selection strategy.


In [ ]:
import random
import numpy as np
from collections import deque
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import SGD, Adam

class DQNAgent:
    def __init__(self, state_size, action_space,memory_size=3000):
        # Initialize instance variables
        self.state_size = state_size
        self.action_space = action_space
        self.action_size = action_space.nvec.prod()
        self.memory = deque(maxlen=memory_size)
        self.gamma = 0.95
        self.epsilon = 1.0
        self.epsilon_min = 0.01
        self.epsilon_decay = 0.995
        self.learning_rate = 0.01
        self.model = self._build_model()

    def _build_model(self):
        # Build the neural network model
        model = Sequential()
        model.add(Dense(64, input_dim=self.state_size))
        model.add(Dense(128, activation='relu'))
        model.add(Dense(256, activation='relu'))
        model.add(Dense(128, activation='relu'))
        model.add(Dense(self.action_size, activation='relu'))
        model.compile(loss='mse', optimizer=Adam(lr=self.learning_rate))
        return model

    def remember(self, state, action, reward, next_state, done):
        # Store the experience in memory
        self.memory.append((state, action, reward, next_state, done))

    def act(self, state):
        # Choose an action using the epsilon-greedy strategy
        if np.random.rand() <= self.epsilon:
            return (random.randrange(self.action_space.nvec[0]), random.randrange(self.action_space.nvec[1]))
        act_values = self.model.predict(state)
        return np.unravel_index(np.argmax(act_values[0]), self.action_space.nvec)

    def replay(self, batch_size):
        # Train the model using random samples from memory
        minibatch = random.sample(self.memory, batch_size)
        for state, action, reward, next_state, done in minibatch:
            target = reward
            if not done:
                # Update target value for non-terminal states
                target = reward + self.gamma * np.amax(self.model.predict(next_state)[0])
            target_f = self.model.predict(state)

            # Convert the action tuple to a linear index
            action_idx = np.ravel_multi_index(action, self.action_space.nvec)

            # Update the target value for the chosen action
            target_f[0][action_idx] = target
            self.model.fit(state, target_f, epochs=1, verbose=0)

        # Decay the exploration rate if it's above the minimum threshold
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay


## Step 4: Train the Agent

In this step, we train a `DQNAgent` in the `StockTradingEnv` using a deep Q-learning algorithm. The training process consists of the following steps: 


<div style="display: flex; align-items: center;">
  <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMSkillsNetwork-GPXX0SQEN/_84d78f91-ffa6-43b7-ac39-f3b229beac78.jpeg" width="30%" alt="Image" style="padding-right: 10px;">



1. Load the stock training data and create a StockTradingEnv using the data. Define the state size and action size based on the environment's observation space and action space. 
2. Create a DQNAgent using the state size and action space from the environment. Set the number of training episodes and the batch size for replaying experiences. 
3. Loop through the episodes and reset the environment at the beginning of each episode. Reshape the initial state to match the input format expected by the agent's neural network. Initialize a variable to keep track of the total reward for the episode. 
4. Within each episode, the agent selects an action based on its current state and the epsilon-greedy exploration strategy. The agent then performs the action in the environment and receives the next state, reward, and a flag indicating whether the episode has ended. 
5. The agent stores the experience (state, action, reward, next_state, done) in its memory buffer and updates its current state to the next state. 
6. At the end of each episode, print the episode number, total reward, and current epsilon value. If the agent's memory buffer has enough experiences, perform a replay step to update the neural network weights using a batch of sampled experiences.

    
</div>
<br>

The training process allows the agent to learn a policy for buying and selling stocks by interacting with the environment, updating its neural network, and balancing exploration and exploitation with the epsilon-greedy action selection strategy.


In [ ]:
# Load the stock data
data = train_data

# Create the environment
env = StockTradingEnv(data)
state_size = env.observation_space.shape[0]
action_size = env.action_space.nvec.prod()

# Create and train the agent
agent = DQNAgent(state_size, env.action_space)
episodes = 100
batch_size = 32

for e in tqdm(range(episodes)):
    state = env.reset()
    state = np.reshape(state, [1, state_size])
    total_reward = 0

    while True:
        action = agent.act(state)
        next_state, reward, done, _ = env.step(action)
        total_reward += reward
        next_state = np.reshape(next_state, [1, state_size])

        agent.remember(state, action, reward, next_state, done)
        state = next_state

        if done:
            # print(f'Episode: {e + 1}/{episodes}, Total Reward: {total_reward}, Epsilon: {agent.epsilon:.2}')
            break

    if len(agent.memory) > batch_size:
        agent.replay(batch_size)


## Step 5: Test the agent

Test the agent on a separate dataset to evaluate its performance. To test the agent on a separate dataset, we can create a new instance of the StockTradingEnv with the new dataset and use the trained agent to make decisions.


In [ ]:
import matplotlib.pyplot as plt

# Load the test data
test_data = test_data.iloc[:,:]

# change this to 1 to get records of all transactions
verbose = 0

# Create the test environment
test_env = StockTradingEnv(test_data, initial_cash=10000)

# Test the agent
state = test_env.reset()
state = np.reshape(state, [1, state_size])
total_reward = 0

# Initialize lists to store the buy and sell prices and the corresponding time steps
buy_prices = []
buy_times = []
sell_prices = []
sell_times = []

# Initialize the list to store portfolio values and add the initial portfolio value
portfolio_values = [test_env.cash]

while True:
    action = agent.act(state)
    next_state, reward, done, _ = test_env.step(action)

    # Calculate the total portfolio value
    portfolio_value = test_env.cash + (test_env.shares_held * test_env.data['Adj Close'][test_env.current_step])
    portfolio_values.append(portfolio_value)

    # Print the action details
    action_type, shares = action
    if(verbose ==1):
        if action_type == 0:
            print(f"Buy {shares} shares at price {test_env.data['Adj Close'][test_env.current_step]}")
        elif action_type == 1:
            print(f"Sell {shares} shares at price {test_env.data['Adj Close'][test_env.current_step]}")

        # Print available cash, number of shares held, and reward
        print(f"Available cash: {test_env.cash}",f",Shares held: {test_env.shares_held}",f",Reward: {reward}")
        print("------------------------------------------------")

    total_reward += reward
    next_state = np.reshape(next_state, [1, state_size])
    state = next_state
    
    # Check if a buy action was taken and record the price and time step
    if action[0] == 0:
        buy_prices.append(test_data['Adj Close'][test_env.current_step])
        buy_times.append(test_data.index[test_env.current_step])

    # Check if a sell action was taken and record the price and time step
    elif action[0] == 1:
        sell_prices.append(test_data['Adj Close'][test_env.current_step])
        sell_times.append(test_data.index[test_env.current_step])

    if done:
        print(f'Test: Total Reward: {total_reward}')
        break

# Calculate the percentage change in the portfolio value
portfolio_percentage_changes = [((value - portfolio_values[0]) / portfolio_values[0]) * 100 for value in portfolio_values]

# Create the figure with two subplots
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(10, 12), sharex=True)

# Plot the historical price data as a line chart on the first subplot
ax1.plot(test_data.index, test_data['Adj Close'])

# Plot the buy and sell actions as red and blue dots, respectively, on the first subplot
ax1.scatter(buy_times, buy_prices, color='b', marker='o', label='Buy')
ax1.scatter(sell_times, sell_prices, color='r', marker='o', label='Sell')

# Set the first subplot's title, x-label, and y-label
ax1.set_title('Trading History')
ax1.set_xlabel('Time')
ax1.set_ylabel('Price')
ax1.legend()

# Plot the percentage change in the portfolio value on the second subplot
ax2.plot(test_data.index, portfolio_percentage_changes, color='g', linestyle='--', label='Portfolio % Change')

# Set the second subplot's title, x-label, and y-label
ax2.set_title('Portfolio Value Change')
ax2.set_xlabel('Time')
ax2.set_ylabel('% Change')
ax2.legend()

# Display the chart
plt.show()


___Run the test again and you will get different result.___


<center> <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMSkillsNetwork-GPXX0SQEN/images/meme_honest_work.jpeg" width="40%" alt="Restart kernel"> </center>


When testing the DQN agent, you may observe different results each time due to the agent's exploration strategy, which is governed by the epsilon-greedy policy. The agent uses an exploration rate (`epsilon`) to balance between exploration (trying new actions) and exploitation (using the current best-known action). The agent selects random actions with probability `epsilon`, and as long as `epsilon` is greater than zero, there will be some degree of randomness in the agent's decisions.

If you want your agent to act deterministically during testing, you can set the `epsilon` value to zero, which forces the agent to always choose the best-known action based on its current knowledge without any exploration. To do this, you can add a line of code to set `epsilon` to zero before testing:

```python

agent.epsilon = 0
```

By doing this, your agent will act deterministically, and you should get the same results each time you test it. Keep in mind that doing this will prevent the agent from exploring new actions, which might be suboptimal if the agent hasn't fully learned the optimal policy during training.


### Monte Carlo for Testing


| <img src="https://upload.wikimedia.org/wikipedia/commons/thumb/8/84/Pi_30K.gif/440px-Pi_30K.gif" width="300" alt="human analysis"> |
|:--:| 
| *Monte Carlo method applied to approximating the value of π.* [Wikipedia](https://en.wikipedia.org/wiki/Monte_Carlo_method?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkGuidedProjectsIBMSkillsNetworkGPXX0SQEN2504-2023-01-01#:~:text=Monte%20Carlo%20methods%2C%20or%20Monte,might%20be%20deterministic%20in%20principle.) |



For better testing we utilize Monte Carlo method. the concept of Monte Carlo refers to running multiple simulations or tests of the trained agent to estimate the expected performance of the agent. Each run or simulation uses the same agent but with different starting conditions, which are determined by the initial state provided by the environment. This is particularly useful when dealing with environments that have some level of randomness or stochasticity, as it helps provide a more accurate and robust estimate of the agent's performance.

By running multiple tests (Monte Carlo runs), we are essentially exploring various possible scenarios in the stock trading environment. The agent's decisions and the subsequent rewards obtained can vary across different runs due to differences in the initial state or changes in the agent's exploration-exploitation trade-off as a result of its epsilon-greedy strategy.

After all the Monte Carlo runs are completed, we calculate the average reward over all runs. This average reward gives us a more reliable estimate of the agent's performance than the reward from a single test run, as it captures the agent's behavior across various scenarios. The higher the number of Monte Carlo runs, the more accurate and reliable the estimate of the agent's performance becomes, as it converges to the true expected value.


In [ ]:
# Load the test data
test_data = test_data

# Create the test environment
test_env = StockTradingEnv(test_data,initial_cash =100000)

# Define the number of Monte Carlo runs
monte_carlo_runs = 200
rewards = []

for run in tqdm(range(monte_carlo_runs)):
    # Test the agent
    state = test_env.reset()
    state = np.reshape(state, [1, state_size])
    total_reward = 0

    while True:
        action = agent.act(state)
        next_state, reward, done, _ = test_env.step(action)
        total_reward += reward
        next_state = np.reshape(next_state, [1, state_size])
        state = next_state

        if done:
            # print(f'Test Run: {run + 1}/{monte_carlo_runs}, Total Reward: {total_reward}')
            rewards.append(total_reward)
            break

# Calculate and print the average reward
average_reward = np.mean(rewards)
print(f'Average Reward after {monte_carlo_runs} runs: {average_reward}')


<center> <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMSkillsNetwork-GPXX0SQEN/images/meme_not_bad2.jpeg" width="40%" alt="Obama"> </center>


## Step 6: Refine the model


Refining a reinforcement learning model involves adjusting its parameters to improve its performance. Here are some common ways to refine a model: 


<div style="display: flex; align-items: center;">
  <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMSkillsNetwork-GPXX0SQEN/images/_5b34b14b-bc6f-4955-b81b-6014f3d08950.jpeg" width="30%" alt="Image" style="padding-right: 10px;">

    
1. Adjust the hyperparameters of the reinforcement learning algorithm: Most reinforcement learning algorithms have several hyperparameters, such as the learning rate, discount factor, and exploration rate, that can be adjusted to improve performance. Experiment with different hyperparameters to find the optimal values. 
2. Adjust the reward function: The reward function determines the agent's objective, so it can have a significant impact on its behavior. Consider adjusting the reward function to better incentivize the desired behavior. 
3. Adjust the state space: The state space determines the information that the agent has access to when making decisions. Adding or removing features from the state space can improve the agent's ability to make accurate predictions. 
4. Collect more training data: Training a reinforcement learning model can be data-intensive, so collecting more training data can improve its performance. Consider increasing the amount of training data or collecting data from a wider range of scenarios. 
5. Change the architecture of the neural network: Many reinforcement learning algorithms use neural networks to estimate values or policies. Changing the architecture of the neural network can sometimes improve performance. 
6. Use a different reinforcement learning algorithm: There are many different reinforcement learning algorithms, each with its own strengths and weaknesses. Consider trying a different algorithm if the current one is not performing well.

</div>


## Practice: Adding More Data for Training

We can load more data and train the model with extra data.

Lets have a quick look to more data:


In [ ]:
# getting more data
tickers = ["GOOG","IBM","AAPL","META","AMZN"]  #"^GSPC" S&P500
df = yf.download(tickers = tickers ,       # list of tickers
                  period = "2y",         # time period
                  interval = "1h",       # trading interval
                  ignore_tz = True,      # ignore timezone when aligning data from different exchanges?
                  prepost = False)       # download pre/post market hours data?  
print("shape of dataset: ", df.shape)
df.head()

### plotting the Adj Close for the 5 companies

In the code below, loop through each stock in the 'Adj Close' column and plot.


In [ ]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

split_line = int(np.round(len(df) * 0.1))  # use 10% of data for testing

# Get min and max of 'Adj Close'
adj_close_min = df['Adj Close'].min().min()
adj_close_max = df['Adj Close'].max().max()

# Set the background color
sns.set_style(rc={'axes.facecolor': 'lightsteelblue'})
plt.figure(figsize=(12, 4))

# Loop through each stock in the 'Adj Close' column and plot
for stock in df['Adj Close'].columns:
    sns.lineplot(data=df['Adj Close'][stock], label=f'Adjusted Close Price - {stock}')

plt.xticks(rotation=45)
plt.title(', '.join(df['Adj Close'].columns))

# single vline with full ymin and ymax
plt.vlines(x=df.index[-split_line], ymin=adj_close_min, ymax=adj_close_max, colors='green', ls=':', lw=2, label='Train-Test Split Point')

plt.legend(bbox_to_anchor=(1.0, 1), loc='upper left')
plt.show()


<details>
    <summary>Click here for Solution</summary>

```python
for stock in df['Adj Close'].columns:
    sns.lineplot(data=df['Adj Close'][stock], label=f'Adjusted Close Price - {stock}')
```
</details>


### Let's create a function that adds all the indicators to the dataframe

Write `add_indicators` function adds several technical indicators (we had before) to a given DataFrame, and remove `na` and remove columns `Open`, `High`, `Low`, `Close`, `Volume`.


In [ ]:
def add_indicators(df):

    df['EMA7'] = ta.trend.EMAIndicator(close= df['Adj Close'], window= 7, fillna= False).ema_indicator()
    df['EMA14'] = ta.trend.EMAIndicator(close= df['Adj Close'], window= 14, fillna= False).ema_indicator()
    df['EMA50'] = ta.trend.EMAIndicator(close= df['Adj Close'], window= 50, fillna= False).ema_indicator()
    df['EMA200'] = ta.trend.EMAIndicator(close= df['Adj Close'], window= 50, fillna= False).ema_indicator()
    df['MACD_line'] =ta.trend.MACD(close= df['Adj Close'], window_slow= 26, window_fast= 12, window_sign = 9, fillna = False).macd()
    df['MACD_signal'] =ta.trend.MACD(close= df['Adj Close'], window_slow= 26, window_fast= 12, window_sign = 9, fillna = False).macd_signal()
    df['MACD_diff'] =ta.trend.MACD(close= df['Adj Close'], window_slow= 26, window_fast= 12, window_sign = 9, fillna = False).macd_diff()
    df['RSI']=ta.momentum.RSIIndicator(close= df['Adj Close'], window= 14, fillna= False).rsi()
    df['OBV']= ta.volume.OnBalanceVolumeIndicator(close= df['Adj Close'], volume= df['Volume'], fillna = False).on_balance_volume()
    df['BBH']=ta.volatility.BollingerBands(df['Adj Close'], window = 20, window_dev = 2, fillna = False).bollinger_hband_indicator()
    df['BBL']=ta.volatility.BollingerBands(df['Adj Close'], window = 20, window_dev = 2, fillna = False).bollinger_lband_indicator()
    df=df.dropna()
    df = df.drop(columns=['Open', 'High', 'Low', 'Close', 'Volume'],axis=1)
    
    return df

<details>
    <summary>Click here for Solution</summary>

```python
def add_indicators(df):

    df['EMA7'] = ta.trend.EMAIndicator(close= df['Adj Close'], window= 7, fillna= False).ema_indicator()
    df['EMA14'] = ta.trend.EMAIndicator(close= df['Adj Close'], window= 14, fillna= False).ema_indicator()
    df['EMA50'] = ta.trend.EMAIndicator(close= df['Adj Close'], window= 50, fillna= False).ema_indicator()
    df['EMA200'] = ta.trend.EMAIndicator(close= df['Adj Close'], window= 50, fillna= False).ema_indicator()
    df['MACD_line'] =ta.trend.MACD(close= df['Adj Close'], window_slow= 26, window_fast= 12, window_sign = 9, fillna = False).macd()
    df['MACD_signal'] =ta.trend.MACD(close= df['Adj Close'], window_slow= 26, window_fast= 12, window_sign = 9, fillna = False).macd_signal()
    df['MACD_diff'] =ta.trend.MACD(close= df['Adj Close'], window_slow= 26, window_fast= 12, window_sign = 9, fillna = False).macd_diff()
    df['RSI']=ta.momentum.RSIIndicator(close= df['Adj Close'], window= 14, fillna= False).rsi()
    df['OBV']= ta.volume.OnBalanceVolumeIndicator(close= df['Adj Close'], volume= df['Volume'], fillna = False).on_balance_volume()
    df['BBH']=ta.volatility.BollingerBands(df['Adj Close'], window = 20, window_dev = 2, fillna = False).bollinger_hband_indicator()
    df['BBL']=ta.volatility.BollingerBands(df['Adj Close'], window = 20, window_dev = 2, fillna = False).bollinger_lband_indicator()
    df=df.dropna()
    df = df.drop(columns=['Open', 'High', 'Low', 'Close', 'Volume'],axis=1)
    
    return df
```
</details>


Now, create separate data frame for each stock data.


In [ ]:
# loading individual stock data for every stock and calling add_indicator

df_GOOG = yf.download(tickers = 'GOOG' ,period = "2y", interval = "1h",ignore_tz = True,prepost = False) 
df_GOOG = add_indicators(df_GOOG)

df_IBM = yf.download(tickers = 'IBM' ,period = "2y", interval = "1h",ignore_tz = True,prepost = False) 
df_IBM = add_indicators(df_IBM)

df_AAPL = yf.download(tickers = 'AAPL' ,period = "2y", interval = "1h",ignore_tz = True,prepost = False) 
df_AAPL = add_indicators(df_AAPL)

df_META = yf.download(tickers = 'META' ,period = "2y", interval = "1h",ignore_tz = True,prepost = False) 
df_META = add_indicators(df_META)

df_AMZN = yf.download(tickers = 'AMZN' ,period = "2y", interval = "1h",ignore_tz = True,prepost = False) 
df_AMZN = add_indicators(df_AMZN)

train_data1 = df_GOOG.iloc[:-split_line,:]
train_data2 = df_IBM.iloc[:-split_line,:]
train_data3 = df_AAPL.iloc[:-split_line,:]
train_data4 = df_META.iloc[:-split_line,:]
train_data5 = df_AMZN.iloc[:-split_line,:]

train_df_list = [train_data1,train_data2,train_data3,train_data4,train_data5]

### Exercise: optimize the above code
You can optimize the code by using a loop and a list of stock tickers. In the optimized code, we store the stock tickers in a list called `tickers`. We then loop through the list, downloading the stock data and adding indicators for each ticker. Finally, we split the data and add it to the `train_df_list`. This reduces the amount of repetitive code and makes it easier to add or remove stocks by simply modifying the `tickers` list.


In [ ]:
import yfinance as yf

tickers = ['GOOG', 'IBM', 'AAPL', 'META', 'AMZN']
train_df_list = []

for ticker in tickers:
    df = yf.download(tickers=ticker, period="2y", interval="1h", ignore_tz=True, prepost=False)
    df = add_indicators(df)
    train_data = df.iloc[:-split_line, :]
    train_df_list.append(train_data)

<details>
    <summary>Click here for Solution</summary>
    
```python

import yfinance as yf

tickers = ['GOOG', 'IBM', 'AAPL', 'META', 'AMZN']
train_df_list = []

for ticker in tickers:
    df = yf.download(tickers=ticker, period="2y", interval="1h", ignore_tz=True, prepost=False)
    df = add_indicators(df)
    train_data = df.iloc[:-split_line, :]
    train_df_list.append(train_data)
```
    
</details>


### Training the Model with More Data


In [ ]:
'''
training loop to handle multiple dataframes like df_1, df_2, df_3, you can create a list of dataframes and loop through them. 
'''
# Load the stock dataframes
data_list = train_df_list

# Create and train the agent
agent = DQNAgent(state_size, env.action_space, memory_size=4000) # increasing the memory size for the agent 
episodes = 100
batch_size = 32

for data in data_list:
    env = StockTradingEnv(data)
    state_size = env.observation_space.shape[0]
    action_size = env.action_space.nvec.prod()

    for e in tqdm(range(episodes)):
        state = env.reset()
        state = np.reshape(state, [1, state_size])
        total_reward = 0

        while True:
            action = agent.act(state)
            next_state, reward, done, _ = env.step(action)
            total_reward += reward
            next_state = np.reshape(next_state, [1, state_size])

            agent.remember(state, action, reward, next_state, done)
            state = next_state

            if done:
                # print(f'Episode: {e + 1}/{episodes}, Total Reward: {total_reward}, Epsilon: {agent.epsilon:.2}')
                break

        if len(agent.memory) > batch_size:
            agent.replay(batch_size)


### Testing the Data

let's split the Apple stock data using `split_line` , that we defined before, for testing purpose.


In [ ]:
# using Apple data for testing
# write your code here



# Create the test environment
test_env = StockTradingEnv(test_data,initial_cash =10000)

# Define the number of Monte Carlo runs
monte_carlo_runs = 200
rewards = []

for run in tqdm(range(monte_carlo_runs)):
    # Test the agent
    state = test_env.reset()
    state = np.reshape(state, [1, state_size])
    total_reward = 0

    while True:
        action = agent.act(state)
        next_state, reward, done, _ = test_env.step(action)
        total_reward += reward
        next_state = np.reshape(next_state, [1, state_size])
        state = next_state

        if done:
            # print(f'Test Run: {run + 1}/{monte_carlo_runs}, Total Reward: {total_reward}')
            rewards.append(total_reward)
            break

# Calculate and print the average reward
average_reward = np.mean(rewards)
print(f'Average Reward after {monte_carlo_runs} runs: {average_reward}')


<details>
    <summary>Click here for Solution</summary>
    
```python  

test_data = df_AAPL.iloc[-split_line:,:]

```
</details>


 
 ### 💡 Some tips to increase the performance. 
 Here are a few modifications you can make to improve the performance of your model: 
- Normalize the input data: Scaling the input features can help your neural network to learn more effectively. Add a function in your `StockTradingEnv` class to normalize the input data, and apply it to your observation data:

```python

from sklearn.preprocessing import MinMaxScaler

class StockTradingEnv(gym.Env):
    # ...

    def _normalize_data(self, data):
        scaler = MinMaxScaler()
        return scaler.fit_transform(data)

    def _get_obs(self):
        # Calculate the observation vector for the current step
        obs = np.array([
            # ...
        ])

        # Normalize the observation vector
        obs = self._normalize_data(obs)

        # Return the observation vector
        return obs
```

 
- Add more layers or increase the number of neurons in the neural network: A more complex model can potentially capture more intricate patterns in the data. However, keep in mind that increasing the complexity of the model might also increase the risk of overfitting. Modify the `_build_model` method in your `DQNAgent` class:

```python

def _build_model(self):
    # Build the neural network model
    model = Sequential()
    model.add(Dense(128, input_dim=self.state_size, activation='relu'))
    model.add(Dense(128, activation='relu'))
    model.add(Dense(256, activation='relu'))
    model.add(Dense(512, activation='relu'))
    model.add(Dense(128, activation='relu'))
    model.add(Dense(self.action_size, activation='linear'))
    model.compile(loss='mse', optimizer=Adam(lr=self.learning_rate))
    return model
```

 
- Increase the memory size: A larger memory size can potentially help your agent remember more experiences and learn from them. Update the memory size in your `DQNAgent` class:

```python

class DQNAgent:
    def __init__(self, state_size, action_space):
        # ...
        self.memory = deque(maxlen=5000)
        # ...
```

 
- Adjust hyperparameters: Experiment with different values for `gamma`, `epsilon_decay`, and `learning_rate` in your `DQNAgent` class. There is no one-size-fits-all solution for these hyperparameters, and you may need to try different combinations to find the best values for your specific problem.

Remember that even with these modifications, it's important to test and validate the model on different datasets and time periods to ensure it generalizes well to unseen data. Also, note that the stock market is inherently unpredictable, and even a well-trained model may not always produce profitable results.


## Authors


|  |  |
|---|---|
| **Sina Nazeri**  | [linkedin](https://www.linkedin.com/in/sina-nazeri?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkGuidedProjectsIBMSkillsNetworkGPXX0SQEN2504-2023-01-01) |
| ![Sina](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMSkillsNetwork-GPXX0Q8REN/images/sina_low_qual.jpeg) | As a data scientist in IBM, I have always been passionate about sharing my knowledge and helping others learn about the field. I believe that everyone should have the opportunity to learn about data science, regardless of their background or experience level. This belief has inspired me to become a learning content provider, creating and sharing educational materials that are accessible and engaging for everyone. |


### Other Contributors


Joseph Santarcangelo

J.C.(Junxing) Chen 

Sheng-Kai Chen 

Artem Arutyunov 

Justin Correia 

Roodra Kanwar 

Vicky Kuo


## Change Log


|Date (YYYY-MM-DD)|Version|Changed By|Change Description|
|-|-|-|-|
|2023-06-01|0.1|Sina Nazeri|Create Lab Template|


Copyright © 2022 IBM Corporation. All rights reserved.
